The Notebook should be run from the top to bottom. The interface will pop up once the final block of code is executed. 

In [1]:
!pip install gradio==3.40.1
import torch
import gradio as gr
from torch import nn, optim
import torchtext
from torchtext import data
import torch.nn.functional as F

In [2]:
class BaselineModel(torch.nn.Module):
    def __init__(self, vocab):
        super().__init__()

        self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
        self.linear = nn.Linear(len(vocab.vectors[0]), 1)

    def forward(self, x):
        # x shape: (batch_size, word_indicies_len)
        # get the embeddings first
        embeddings = self.embedding(x)
        # embedding shape: (batch_size, word_indicies_len, embedding_size)
        average = torch.mean(embeddings, dim=1) # take the average of all the word embeddings in the sentence
        # average shape: (batch_size, embedding_size)
        prediction = self.linear(average)
        return prediction

class CNNModel(torch.nn.Module):
    def __init__(self, vocab, k1, k2, n1, n2, freeze=True):
        super().__init__()

        self.embedding = nn.Embedding.from_pretrained(vocab.vectors, freeze=freeze)
        embedding_size = len(vocab.vectors[0])
        # 1st convolution layer  parameters(in_kernels, out_feature_maps, kernel_size)
        # 1 input matrix, n1 output feature maps (there are n1 kernels)
        self.conv_1 = nn.Conv2d(1, n1, (k1, embedding_size), bias=False)
        # 2nd convolution layer
        self.conv_2 = nn.Conv2d(1, n2, (k2, embedding_size), bias=False)
        # final linear layer
        self.linear = nn.Linear(n1 + n2, 1)

    def forward(self, x):
        # x shape: (batch_size, sentence_len/word_indices_len(N))

        # nn.Conv2d's input tensor should have the shape (batch_size, num_in_kernals, height_in, width_in)
        # nn.Conv2d's input tensor should have the shape
        # (batch_size, num_out_feature_maps, height_feature_map, width_feature_map)

        # embedding shape: (batch_size, N, embedding_size)
        # after squeeze: (batch_size, 1, N, embedding_size)
        x = self.embedding(x).unsqueeze(1)
        # apply activation function after two convolutional layer
        x1 = F.relu(self.conv_1(x)) # output shape: (batch_size, n1, N-k1+1, 1)
        x2 = F.relu(self.conv_2(x)) # output shape: (batch_size, n2, N-k2+1, 1)
        # apply maxpool over the output feature maps, over the N-k1/k2+1 values
        pool_1 = nn.MaxPool1d(x1.shape[2])
        pool_2 = nn.MaxPool1d(x2.shape[2])
        # maxpool input: (batch_size, num_of_channels, sequence_length)
        output_1 = pool_1(x1.squeeze(-1)).squeeze(-1) # output shape: (batch_size, n1)
        output_2 = pool_2(x2.squeeze(-1)).squeeze(-1) # output shape: (batch_size, n2)
        output = torch.cat((output_1, output_2), dim=1) # output shape: (batch_size, n1+n2)
        prediction = self.linear(output)
        return prediction


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
glove = torchtext.vocab.GloVe(name="6B",dim=100)
checkpoint_baseline = torch.load("model_baseline.pt", map_location=device)
baseline_model = BaselineModel(glove)
baseline_model.load_state_dict(checkpoint_baseline)
checkpoint_cnn = torch.load('model_cnn.pt')
cnn_model = CNNModel(glove, k1=2, k2=4, n1=100, n2=100)
cnn_model.load_state_dict(checkpoint_cnn)

<All keys matched successfully>

In [ ]:
def classify(sentence):
    tokens = sentence.split() # Convert to integer representation per token
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    # Convert into a tensor of the shape accepted by the models
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    token_tensor = token_tensor.to(torch.int32)
    token_tensor = torch.transpose(token_tensor, 0, 1)

    baseline_output = baseline_model.forward(token_tensor)
    baseline_output = torch.sigmoid(baseline_output)
    if (baseline_output > 0.5):
      baseline_output = "It's a subjective sentence with {:.4f} probability".format(baseline_output.item())
    else:
      baseline_output = "It's an objective sentence with {:.4f} probability".format(baseline_output.item())

    # if input sentence is smaller than the max kernel size, pad with 0
    max_kernel_size = max(cnn_model.conv_1.kernel_size[0], cnn_model.conv_2.kernel_size[0])
    if len(token_tensor) < max_kernel_size:
        zeros = torch.zeros(1, max_kernel_size - len(token_tensor), dtype=torch.int32)
        token_tensor = torch.cat((token_tensor, zeros), dim=1)
    cnn_output = cnn_model.forward(token_tensor)
    cnn_output = torch.sigmoid(cnn_output)
    if (cnn_output > 0.5):
      cnn_output = "It's a subjective sentence with {:.4f} probability".format(cnn_output.item())
    else:
      cnn_output = "It's an objective sentence with {:.4f} probability".format(cnn_output.item())

    return baseline_output, cnn_output

demo = gr.Interface(
    fn=classify,
    inputs="text",
    outputs=["text", "text"],
)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ff06ba0b8dd4191bc9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
